# 即時讀取用戶語音，轉換為文字 
1. 固定語音秒數

In [2]:
"""PyAudio Example: Record a few seconds of audio and save to a wave file."""


import os
from google.cloud import speech, storage

import wave
import sys
import pyaudio

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1 if sys.platform == 'darwin' else 2
RATE = 44100
RECORD_SECONDS = 5 #設定為5秒

with wave.open('output.wav', 'wb') as wf:
    p = pyaudio.PyAudio()
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)

    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True) #聲音讀取

    print('Recording...')
    for _ in range(0, RATE // CHUNK * RECORD_SECONDS):
        wf.writeframes(stream.read(CHUNK))
    print('Done')

    stream.close()
    p.terminate()
    
YOUR_SERVICE = 'key.json'  #金鑰的檔名
YOUR_AUDIO = 'output.wav'   #音檔的檔名
YOUR_BUCKET = 'pennyvoice' #GCP上bucket的名字

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = YOUR_SERVICE
storage_client = storage.Client()
speech_client = speech.SpeechClient()

# upload file to GCS(Google Cloud Storage)
bucket = storage_client.bucket(YOUR_BUCKET)
bucket.blob(YOUR_AUDIO).upload_from_filename(YOUR_AUDIO)
uri = f'gs://{YOUR_BUCKET}/{YOUR_AUDIO}'

# transcript the audio
audio = speech.RecognitionAudio(uri=uri)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,  # MP3 is under beta
    sample_rate_hertz=44100,
    audio_channel_count=2,  # take care, default is 1
    language_code="zh-TW",
    max_alternatives=10
)
response = speech_client.recognize(config=config, audio=audio)

for r in response.results:
    print(f'{r.alternatives[0].transcript}, {r.alternatives[0].confidence:.3f}')

Recording...
Done
吃飽了嗎, 0.926


# 即時讀取用戶語音，轉換為文字
2. 任意語音秒數 (optional),輸入" 任一鍵" 隨時可停止錄音
<!-- Hint
input() feat. threading -->

In [3]:
"""PyAudio Example: Record a few seconds of audio and save to a wave file."""

import threading
import keyboard
import os
from google.cloud import speech, storage

import wave
import sys
import pyaudio

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1 if sys.platform == 'darwin' else 2
RATE = 44100
RECORD_SECONDS = 5 #設定為5秒
flag = True


def stop():
    global flag #要連接到def外的flag=True
    input("Any key to stop recording.")
    flag = False

with wave.open('output.wav', 'wb') as wf:
    p = pyaudio.PyAudio()
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)

    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True) #聲音讀取

    print('Recording...')
    
    thread = threading.Thread(target=stop)
    thread.start()
    
    while flag:
        wf.writeframes(stream.read(CHUNK))
    print('Done')

    stream.close()
    p.terminate()
    
YOUR_SERVICE = 'key.json'  #金鑰的檔名
YOUR_AUDIO = 'output.wav'   #音檔的檔名
YOUR_BUCKET = 'pennyvoice' #GCP上bucket的名字

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = YOUR_SERVICE
storage_client = storage.Client()
speech_client = speech.SpeechClient()

# upload file to GCS(Google Cloud Storage)
bucket = storage_client.bucket(YOUR_BUCKET)
bucket.blob(YOUR_AUDIO).upload_from_filename(YOUR_AUDIO)
uri = f'gs://{YOUR_BUCKET}/{YOUR_AUDIO}'

# transcript the audio
audio = speech.RecognitionAudio(uri=uri)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,  # MP3 is under beta
    sample_rate_hertz=44100,
    audio_channel_count=2,  # take care, default is 1
    language_code="zh-TW",
    max_alternatives=10
)
print("Transcribing audio...")
response = speech_client.recognize(config=config, audio=audio)

for r in response.results:
    print(f'{r.alternatives[0].transcript}, {r.alternatives[0].confidence:.3f}')

Recording...
Any key to stop recording.u
Done
Transcribing audio...
哈囉今天天氣真好太陽好大好了啦, 0.926


# 即時讀取用戶語音，轉換為文字
2. 任意語音秒數 (optional),輸入"空白鍵" 隨時可停止錄音

In [4]:
import os
from google.cloud import speech, storage

import wave
import sys

import pyaudio
import keyboard

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1 if sys.platform == 'darwin' else 2
RATE = 44100

p = pyaudio.PyAudio()
stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

print('Recording... Press space to stop.')

frames = []
while True:
    data = stream.read(CHUNK)
    frames.append(data)
    if keyboard.is_pressed(' '):
        break

print('Done recording.')

stream.stop_stream()
stream.close()
p.terminate()

with wave.open('output.wav', 'wb') as wf:
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))

YOUR_SERVICE = 'key.json'  # 金鑰的檔名
YOUR_AUDIO = 'output.wav'   # 音檔的檔名
YOUR_BUCKET = 'pennyvoice'  # GCP上bucket的名字

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = YOUR_SERVICE
storage_client = storage.Client()
speech_client = speech.SpeechClient()

# upload file to GCS(Google Cloud Storage)
bucket = storage_client.bucket(YOUR_BUCKET)
blob = bucket.blob(YOUR_AUDIO)
blob.upload_from_filename(YOUR_AUDIO)
uri = f'gs://{YOUR_BUCKET}/{YOUR_AUDIO}'

# transcript the audio
audio = speech.RecognitionAudio(uri=uri)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=RATE,
    audio_channel_count=CHANNELS,
    language_code="zh-TW",
    max_alternatives=10
)
response = speech_client.recognize(config=config, audio=audio)

for r in response.results:
    print(f'{r.alternatives[0].transcript}, {r.alternatives[0].confidence:.3f}')


Recording... Press space to stop.
Done recording.
你好你好歡迎光臨, 0.926
